In [10]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, SVD, NMF
from surprise.model_selection import cross_validate, KFold
from prettytable import PrettyTable
import pickle

# Load the MovieLens 1M dataset
data_info_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings_data = pd.read_csv(r'C:\Users\syamsaiakhil.d\OneDrive - Qentelli\Desktop\Movie Recommendation Project\ml-1m dataset\ratings.dat', sep='::', names=data_info_cols, encoding='latin-1', engine='python')

# Define a Reader object to parse the data
reader = Reader(rating_scale=(1, 5))

# Load the dataset into Surprise's data structure
data = Dataset.load_from_df(ratings_data[['user_id', 'movie_id', 'rating']], reader)

# Load movie titles and genres into a DataFrame
movies_info_cols = ['movie_id', 'title', 'genres']
movie_titles = pd.read_csv(r'C:\Users\syamsaiakhil.d\OneDrive - Qentelli\Desktop\Movie Recommendation Project\ml-1m dataset\movies.dat', sep='::', names=movies_info_cols, encoding='latin-1', engine='python')

# Define models
models = {'SVD': SVD(), 'NMF': NMF()}

# Perform cross-validation with multiple K-folds only once
results = {}
for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    kf = KFold(n_splits=5, random_state=42)
    cv_results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=kf, verbose=True)
    results[model_name] = cv_results
    print("\n" + "=" * 50)  # Add a line between evaluations

# Find the best model based on RMSE and MAE
best_model_name = min(results, key=lambda k: np.mean(results[k]['test_rmse']))
best_model = models[best_model_name]
print(f"Best Model: {best_model_name}")

# Save the best model to a pickle file
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

Evaluating SVD...
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8760  0.8733  0.8714  0.8724  0.8738  0.8734  0.0016  
MAE (testset)     0.6886  0.6854  0.6838  0.6850  0.6853  0.6856  0.0016  
Fit time          8.76    8.55    8.65    8.65    8.63    8.65    0.07    
Test time         1.59    1.01    1.01    1.60    1.06    1.25    0.28    

Evaluating NMF...
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9179  0.9150  0.9162  0.9144  0.9195  0.9166  0.0019  
MAE (testset)     0.7260  0.7227  0.7242  0.7223  0.7259  0.7242  0.0016  
Fit time          15.88   16.30   16.58   15.97   16.23   16.19   0.25    
Test time         0.86    0.87    0.87    1.51    0.85    0.99    0.26    

Best Model: SVD


In [12]:
while True:
    # Replace 'user_id' with the ID of the user you want to recommend movies for
    user_input = input('Enter user_id (type "exit" to exit):')

    if user_input.lower() == 'exit':
        print("Exited...")
        break

    try:
        user_id = int(user_input)

        # Generate recommendations for the user
        movie_list = []
        rated_movie_ids = set(ratings_data[ratings_data['user_id'] == user_id]['movie_id'])
        for movie_id in set(ratings_data['movie_id']) - rated_movie_ids:
            predicted_rating = best_model.predict(user_id, movie_id).est
            movie_list.append((movie_id, predicted_rating))

        # Sort movies by predicted rating in descending order
        movie_list.sort(key=lambda x: x[1], reverse=True)

        # Create a PrettyTable for recommended movies
        recommended_table = PrettyTable()
        recommended_table.field_names = ["Movie ID", "Movie Name", "Genres", "Predicted Rating"]
        num_recommendations = 10
        for movie_id, predicted_rating in movie_list[:num_recommendations]:
            movie_info = movie_titles[movie_titles['movie_id'] == movie_id]
            if not movie_info.empty:
                movie_name = movie_info.iloc[0]['title']
                movie_genres = movie_info.iloc[0]['genres']
                recommended_table.add_row([movie_id, movie_name, movie_genres, predicted_rating])

        # Get the top-rated movies by the user
        top_rated_movies = ratings_data[ratings_data['user_id'] == user_id].sort_values(by='rating', ascending=False).head(10)

        # Create a PrettyTable for top-rated movies
        top_rated_table = PrettyTable()
        top_rated_table.field_names = ["Movie ID", "Movie Name", "Genres", "Rating"]
        for index, row in top_rated_movies.iterrows():
            movie_info = movie_titles[movie_titles['movie_id'] == row['movie_id']]
            if not movie_info.empty:
                movie_name = movie_info.iloc[0]['title']
                movie_genres = movie_info.iloc[0]['genres']
                top_rated_table.add_row([row['movie_id'], movie_name, movie_genres, row['rating']])

        # Print the top-rated movies table
        print("\nTop Rated Movies by the User:")
        print(top_rated_table)

        # Print the recommended movies table
        print("Recommended Movies:")
        print(recommended_table)

    except ValueError:
        print("Invalid input. Please enter a valid user_id or 'exit'.")


Enter user_id (type "exit" to exit):134

Top Rated Movies by the User:
+----------+------------------------------------------+-----------------------------+--------+
| Movie ID |                Movie Name                |            Genres           | Rating |
+----------+------------------------------------------+-----------------------------+--------+
|   3783   |             Croupier (1998)              |         Crime|Drama         |   5    |
|   524    |               Rudy (1993)                |            Drama            |   5    |
|   1429   |    Jackie Chan's First Strike (1996)     |            Action           |   5    |
|   2302   |          My Cousin Vinny (1992)          |            Comedy           |   5    |
|   3481   |           High Fidelity (2000)           |            Comedy           |   5    |
|   3827   |           Space Cowboys (2000)           |        Action|Sci-Fi        |   5    |
|   1772   |        Blues Brothers 2000 (1998)        |    Action|Comedy|M